In [1]:
import requests
import json

In [2]:
A=requests.get('https://opencampus-codex.blockscout.com/api/v2/smart-contracts/0x49D8d8a61A7807a8cC78b42A34FA223014518863')
B=requests.get('https://opencampus-codex.blockscout.com/api/v2/smart-contracts/0x394c399dbA25B99Ab7708EdB505d755B3aa29997')


In [3]:
json.loads(A.content)["message"]

'Not found'

In [47]:
import uuid
def isContract(hash):
    A=requests.get('https://opencampus-codex.blockscout.com/api/v2/smart-contracts/%s'%(hash))
    if A.status_code==200:
        return True
    else:
        return False

def maxValue(num):
    if num<1:
        num=1
    if num > 20:
        num=20
    return int(num)

def jsonTx(Tx):
    AAA=Tx 
    if  'contract_creation' in AAA['tx_types']:
        try:
            Sal={
                "value":float(AAA["value"])/10**18,
                "width":maxValue(float(AAA["value"])/10**18),
                "fee":float(AAA['fee']['value'])/10**18,
                'tx_types':AAA['tx_types'],
                'timestamp':AAA['timestamp'],
                'block':AAA['block'],
                'token_transfers':AAA['token_transfers'],
                "from_ens":AAA['from']['ens_domain_name'],
                "from_wallet":AAA['from']['hash'],
                "from_iscontract":AAA['from']['is_contract'],
                "to_wallet":AAA['created_contract']['hash'],  
                "id" : str(uuid.uuid4())  
            }
        except:
            print("Fail")
        return Sal
    else:
        try:  
            Sal={
            "value":float(AAA["value"])/10**18,
            "fee":float(AAA['fee']['value'])/10**18,
            "width":maxValue(float(AAA["value"])/10**18),
            'tx_types':AAA['tx_types'],
            'timestamp':AAA['timestamp'],
            'block':AAA['block'],
            'token_transfers':AAA['token_transfers'],
            "to_ens":AAA['to']['ens_domain_name'],
            "to_wallet":AAA['to']['hash'],
            "to_iscontract":AAA['to']['is_contract'],
            "from_ens":AAA['from']['ens_domain_name'],
            "from_wallet":AAA['from']['hash'],
            "from_iscontract":AAA['from']['is_contract'],
            "id" : str(uuid.uuid4())
            }
        except:
            print("Fail")
        return Sal

def AllTx(Txs,Out):
    wallets=set()
    for i in json.loads(Txs)['items']:
        Temp=jsonTx(i)
        Out.append(Temp)
        wallets.add(Temp["from_wallet"])
        wallets.add(Temp["to_wallet"])
    return Out,wallets


    
def TxsWallet(wallet):
    Contract=isContract(wallet)
    if Contract:
        return {"message":"Is contract"},0,0
    Out=[]
    towallet=requests.get('https://opencampus-codex.blockscout.com/api/v2/addresses/%s/transactions?filter=to'%(wallet))
    Out,walletsto=AllTx(towallet.content,Out)
    fromwallet=requests.get('https://opencampus-codex.blockscout.com/api/v2/addresses/%s/transactions?filter=from'%(wallet))
    Out,walletsfrom=AllTx(fromwallet.content,Out)
    walletsto.union(walletsfrom)

    return Out,walletsto.union(walletsfrom),Contract

def colorContract(test):
    if test:
        return '#FF5733'
    else:
        return "#000000"
def TxAllWallets(wallet,maxloop=1000):
    wallets=[]
    if isContract(wallet):
        return {"message":"Is contract"}
    Out,newwallets,contract=TxsWallet(wallet)
    wallets.append({"id":wallet,"url":"https://opencampus-codex.blockscout.com/address/"+wallet,"color" : colorContract(isContract(wallet))})
    counter=0
    Tot=len(newwallets)
    Doit=set()
    for aw in newwallets:
        counter=counter+1
        if counter>maxloop-1:
            break
        print("%s de %s"%(counter, Tot))
        Out2,temp,Contract=TxsWallet(aw)
        if aw not in Doit:
            wallets.append({"id":aw,"url":"https://opencampus-codex.blockscout.com/address/"+aw,"color" : colorContract(isContract(aw))})
            Doit.add(aw)
        if temp!=0:
            Out3=Out+Out2
            Out=Out3

    return Out, wallets




In [26]:
isContract("0xE18eA458f28792D90bb2063A0A792d51D310207c")

200


True

In [60]:
TxsWallet("0xE18eA458f28792D90bb2063A0A792d51D310207c")

{'message': 'Is contract'}

In [61]:
TxsWallet("0x49D8d8a61A7807a8cC78b42A34FA223014518863")

[{'value': '2500000000000000000',
  'fee': '54769400000000',
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-24T12:53:50.000000Z',
  'block': 36515,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x49D8d8a61A7807a8cC78b42A34FA223014518863',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0xa482AE7d753b33cB06Fd6872443E902cB6BEe592',
  'from_iscontract': False}]

In [62]:
TxsWallet("0x49D8d8a61A7807a8cC78b42A34FA223014518863")

[{'value': '2500000000000000000',
  'fee': '54769400000000',
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-24T12:53:50.000000Z',
  'block': 36515,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x49D8d8a61A7807a8cC78b42A34FA223014518863',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0xa482AE7d753b33cB06Fd6872443E902cB6BEe592',
  'from_iscontract': False}]

In [67]:
2500000000000000000/10**18
54769400000000/10**18

5.47694e-05

In [71]:
A,B=TxsWallet("0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf")

In [72]:
A

[{'value': 20.0,
  'fee': 5.39465e-05,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-24T07:48:26.000000Z',
  'block': 35726,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False},
 {'value': 10.0,
  'fee': 5.3535e-05,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-20T06:18:30.000000Z',
  'block': 29613,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False},
 {'value': 0.5055432,
  'fee': 5.43579e-05,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-12T10:48:52.000000Z',
  'block': 22696,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3

In [48]:
Aw,Bw=TxAllWallets("0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf",3)

1 de 46
2 de 46
3 de 46


In [49]:
Aw

[{'value': 20.0,
  'fee': 5.39465e-05,
  'width': 20,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-24T07:48:26.000000Z',
  'block': 35726,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False,
  'id': '7815916a-3a4b-4f4f-81dd-8ce13ca5c62c'},
 {'value': 10.0,
  'fee': 5.3535e-05,
  'width': 10,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-20T06:18:30.000000Z',
  'block': 29613,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False,
  'id': 'ce7de8d5-82b7-4e87-ad6f-964d33be1c7c'},
 {'value': 0.5055432,
  'fee': 5.43579e-05,
  'width': 1,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-

In [50]:
Bw

[{'id': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'url': 'https://opencampus-codex.blockscout.com/address/0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'color': '#000000'},
 {'id': '0x0f35b8c3289c1d0a864793fB6E0d0eb2106D9c45',
  'url': 'https://opencampus-codex.blockscout.com/address/0x0f35b8c3289c1d0a864793fB6E0d0eb2106D9c45',
  'color': '#000000'},
 {'id': '0x7FD6d5CF43E491909a83eE88bf383000Ff7FB1b5',
  'url': 'https://opencampus-codex.blockscout.com/address/0x7FD6d5CF43E491909a83eE88bf383000Ff7FB1b5',
  'color': '#000000'},
 {'id': '0x40cF4b0F8C88db0f2dd70F1Fddc289c12Da4e7BE',
  'url': 'https://opencampus-codex.blockscout.com/address/0x40cF4b0F8C88db0f2dd70F1Fddc289c12Da4e7BE',
  'color': '#000000'}]

In [20]:
Aw,Bw=TxAllWallets("0x0d1d649753155e2903e80b89201FFF09E238Eb3B")
Aw

1 de 6
2 de 6
3 de 6
4 de 6
5 de 6
6 de 6


[{'value': 0.1,
  'fee': 5.3535e-05,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-26T11:11:46.000000Z',
  'block': 39993,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x0d1d649753155e2903e80b89201FFF09E238Eb3B',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x2ec8175015Bef5ad1C0BE1587C4A377bC083A2d8',
  'from_iscontract': False},
 {'value': 0.0,
  'fee': 0.0001221063,
  'tx_types': ['contract_call'],
  'timestamp': '2024-08-28T15:53:06.000000Z',
  'block': 44830,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x756F971D1C603f297Ad1020BAf96021a3D9F7AE6',
  'to_iscontract': True,
  'from_ens': None,
  'from_wallet': '0x0d1d649753155e2903e80b89201FFF09E238Eb3B',
  'from_iscontract': False},
 {'value': 0.0,
  'fee': 0.0001226951,
  'tx_types': ['contract_call'],
  'timestamp': '2024-08-28T08:12:58.000000Z',
  'block': 44241,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x34c124f69e4ABd3D6C88F6f190b8e2f336084779',
 

In [21]:
Bw

[{'id': '0x0d1d649753155e2903e80b89201FFF09E238Eb3B',
  'url': 'https://opencampus-codex.blockscout.com/address/0x0d1d649753155e2903e80b89201FFF09E238Eb3B',
  'color': '#000000'},
 {'id': '0xF887B4D3b17C12C86cc917cF72fb8881f866a847',
  'url': 'https://opencampus-codex.blockscout.com/address/0xF887B4D3b17C12C86cc917cF72fb8881f866a847',
  'color': '#FF5733'},
 {'id': '0x756F971D1C603f297Ad1020BAf96021a3D9F7AE6',
  'url': 'https://opencampus-codex.blockscout.com/address/0x756F971D1C603f297Ad1020BAf96021a3D9F7AE6',
  'color': '#FF5733'},
 {'id': '0x34c124f69e4ABd3D6C88F6f190b8e2f336084779',
  'url': 'https://opencampus-codex.blockscout.com/address/0x34c124f69e4ABd3D6C88F6f190b8e2f336084779',
  'color': '#FF5733'},
 {'id': '0x2c51D62d65b8ba191Db7aC3420F213D38c853aFd',
  'url': 'https://opencampus-codex.blockscout.com/address/0x2c51D62d65b8ba191Db7aC3420F213D38c853aFd',
  'color': '#FF5733'},
 {'id': '0x2ec8175015Bef5ad1C0BE1587C4A377bC083A2d8',
  'url': 'https://opencampus-codex.blockscout.

In [45]:
len(AA['items'])

1

In [12]:
AAA=AA['items'][0]

AA['items'][0]

{'timestamp': '2024-08-24T12:53:50.000000Z',
 'fee': {'type': 'actual', 'value': '54769400000000'},
 'gas_limit': '713507',
 'block': 36515,
 'status': 'ok',
 'method': None,
 'confirmations': 2070,
 'type': 2,
 'exchange_rate': None,
 'to': {'ens_domain_name': None,
  'hash': '0x49D8d8a61A7807a8cC78b42A34FA223014518863',
  'implementation_address': None,
  'implementation_name': None,
  'implementations': [],
  'is_contract': False,
  'is_verified': False,
  'metadata': None,
  'name': None,
  'private_tags': [],
  'public_tags': [],
  'watchlist_names': []},
 'tx_burnt_fee': '54769400000000',
 'max_fee_per_gas': '100000000000',
 'result': 'success',
 'hash': '0x4b52d5f444bf6202d823ba73914efbd4ebf9f02f72fa036ad9584ce2ce6da0fe',
 'gas_price': '100000000',
 'priority_fee': '1095388000000000',
 'base_fee_per_gas': '100000000',
 'from': {'ens_domain_name': None,
  'hash': '0xa482AE7d753b33cB06Fd6872443E902cB6BEe592',
  'implementation_address': None,
  'implementation_name': None,
  'impl

In [9]:

import pandas as pd
pd.DataFrame(Aw).columns

Index(['value', 'fee', 'tx_types', 'timestamp', 'block', 'token_transfers',
       'to_ens', 'to_wallet', 'to_iscontract', 'from_ens', 'from_wallet',
       'from_iscontract'],
      dtype='object')

In [59]:
AAA['tx_types']


['coin_transfer']

{'value': '0', 'tx_types': ['contract_creation'], 'timestamp': '2024-08-25T07:53:32.000000Z', 'block': 38078, 'token_transfers': None, 'from_ens': None, 'from_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'from_iscontract': False, 'to_wallet': '0x5E8C933d44b2457D4dA7BC05e0421c2AF00247cA', 'fee': '1493096700000000'}
{'value': '0', 'tx_types': ['contract_creation'], 'timestamp': '2024-08-23T06:56:29.000000Z', 'block': 34529, 'token_transfers': None, 'from_ens': None, 'from_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'from_iscontract': False, 'to_wallet': '0xD39F12a43fD40fdcc4a4881d381Ac9fdbCa3FeAe', 'fee': '1493096700000000'}
{'value': '0', 'tx_types': ['contract_creation'], 'timestamp': '2024-08-22T10:58:50.000000Z', 'block': 33111, 'token_transfers': None, 'from_ens': None, 'from_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'from_iscontract': False, 'to_wallet': '0xc4E9F06c473C5a8B99bA701989fF91AA5C38bf1f', 'fee': '1008031900000000'}
{'value': '0', 'tx_typ

In [ ]:
{'timestamp': '2024-08-25T07:53:32.000000Z', 'fee': {'type': 'actual', 'value': '1493096700000000'}, 'gas_limit': '16662856', 'block': 38078, 'status': 'ok', 'method': None, 'confirmations': 507, 'type': 2, 'exchange_rate': None, 'to': None, 'tx_burnt_fee': '1493096700000000', 'max_fee_per_gas': '100000000', 'result': 'success', 'hash': '0xe75f13946857f06193e300c43efb9606162f0c09250309d6a497308e7ae1dd15', 'gas_price': '100000000', 'priority_fee': '0', 'base_fee_per_gas': '100000000', 'from': {'ens_domain_name': None, 'hash': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'implementation_address': None, 'implementation_name': None, 'implementations': [], 'is_contract': False, 'is_verified': False, 'metadata': None, 'name': None, 'private_tags': [], 'public_tags': [], 'watchlist_names': []}, 'token_transfers': None, 'tx_types': ['contract_creation'], 'gas_used': '14930967', 'created_contract': {'ens_domain_name': None, 'hash': '0x5E8C933d44b2457D4dA7BC05e0421c2AF00247cA', 'implementation_address': None, 'implementation_name': None, 'implementations': [], 'is_contract': True, 'is_verified': False, 'metadata': None, 'name': None, 'private_tags': [], 'public_tags': [], 'watchlist_names': []}, 'position': 1, 'nonce': 74, 'has_error_in_internal_txs': False, 'actions': [], 'decoded_input': None, 'token_transfers_overflow': None, 'raw_input': '0x6080604052348015600e575f80fd5b50335f806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550611ab38061005b5f395ff3fe608060405260043610610089575f3560e01c8063aca38e6d11610058578063aca38e6d1461014b578063bc89fc7214610175578063c6e134c61461019d578063f851a440146101c5578063fedf1801146101ef57610090565b806313efe858146100945780635cbb3a01146100bc57806366852374146100e4578063a6a4e1ab1461012357610090565b3661009057005b5f80fd5b34801561009f575f80fd5b506100ba60048036038101906100b59190610fa8565b610230565b005b3480156100c7575f80fd5b506100e260048036038101906100dd9190611047565b610413565b005b3480156100ef575f80fd5b5061010a60048036038101906101059190610fa8565b6106b6565b60405161011a949392919061113d565b60405180910390f35b34801561012e575f80fd5b5061014960048036038101906101449190611187565b6107ab565b005b348015610156575f80fd5b5061015f6109e3565b60405161016c919061125a565b60405180910390f35b348015610180575f80fd5b5061019b60048036038101906101969190610fa8565b6109e9565b005b3480156101a8575f80fd5b506101c360048036038101906101be9190611273565b610c28565b005b3480156101d0575f80fd5b506101d9610cfa565b6040516101e6919061129e565b60405180910390f35b3480156101fa575f80fd5b5061021560048036038101906102109190611273565b610d1d565b604051610227969594939291906112b7565b60405180910390f35b8160015f8281526020019081526020015f206004015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146102d2576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016102c99061139b565b60405180910390fd5b60025f8481526020019081526020015f205f8373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f2060020160019054906101000a900460ff1661036e576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161036590611403565b60405180910390fd5b600160025f8581526020019081526020015f205f8473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f206002015f6101000a81548160ff0219169083151502179055507fd4829f45099f9fa7e85153a0ea413a85dadd5d09c3ff1baa69160e014c86e4ea8383604051610406929190611421565b60405180910390a1505050565b60015f8481526020019081526020015f2060040160149054906101000a900460ff16610474576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161046b90611492565b60405180910390fd5b60025f8481526020019081526020015f205f3373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f2060020160019054906101000a900460ff1615610511576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161050890611520565b60405180910390fd5b60405180608001604052803373ffffffffffffffffffffffffffffffffffffffff16815260200183838080601f0160208091040260200160405190810160405280939291908181526020018383808284375f81840152601f19601f8201169050808301925050505050505081526020015f151581526020016001151581525060025f8581526020019081526020015f205f3373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f205f820151815f015f6101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060208201518160010190816106359190611765565b506040820151816002015f6101000a81548160ff02191690831515021790555060608201518160020160016101000a81548160ff0219169083151502179055509050507f394b2f46e841d2b670718e98a15dfb76853abc5a8761449c74e9cba62d90152183336040516106a9929190611421565b60405180910390a1505050565b6002602052815f5260405f20602052805f5260405f205f9150915050805f015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff169080600101805461070590611598565b80601f016020809104026020016040519081016040528092919081815260200182805461073190611598565b801561077c5780601f106107535761010080835404028352916020019161077c565b820191905f5260205f20905b81548152906001019060200180831161075f57829003601f168201915b505050505090806002015f9054906101000a900460ff16908060020160019054906101000a900460ff16905084565b6040518060c0016040528088888080601f0160208091040260200160405190810160405280939291908181526020018383808284375f81840152601f19601f82011690508083019250505050505050815260200186868080601f0160208091040260200160405190810160405280939291908181526020018383808284375f81840152601f19601f82011690508083019250505050505050815260200184848080601f0160208091040260200160405190810160405280939291908181526020018383808284375f81840152601f19601f8201169050808301925050505050505081526020018281526020013373ffffffffffffffffffffffffffffffffffffffff1681526020016001151581525060015f60035481526020019081526020015f205f820151815f0190816108e09190611765565b5060208201518160010190816108f69190611765565b50604082015181600201908161090c9190611765565b50606082015181600301556080820151816004015f6101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060a08201518160040160146101000a81548160ff0219169083151502179055509050507fb95f0f805f90a93611b82f20f5f8abf84ef8bafb9146dc64f41bada29a378dad60035488888888336040516109bb9695949392919061186e565b60405180910390a160035f8154809291906109d5906118f0565b919050555050505050505050565b60035481565b8160015f8281526020019081526020015f206004015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614610a8b576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610a829061139b565b60405180910390fd5b60025f8481526020019081526020015f205f8373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f206002015f9054906101000a900460ff16610b26576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610b1d90611981565b60405180910390fd5b5f60015f8581526020019081526020015f206003015490505f8373ffffffffffffffffffffffffffffffffffffffff1682604051610b63906119cc565b5f6040518083038185875af1925050503d805f8114610b9d576040519150601f19603f3d011682016040523d82523d5f602084013e610ba2565b606091505b5050905080610be6576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610bdd90611a2a565b60405180910390fd5b7f8a9c00e0330ad6bdd028b5f904d12736ca2c437d2321f25af05bd5c51cc80c9b858584604051610c1993929190611a48565b60405180910390a15050505050565b8060015f8281526020019081526020015f206004015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614610cca576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610cc19061139b565b60405180910390fd5b5f60015f8481526020019081526020015f2060040160146101000a81548160ff0219169083151502179055505050565b5f8054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b6001602052805f5260405f205f91509050805f018054610d3c90611598565b80601f0160208091040260200160405190810160405280929190818152602001828054610d6890611598565b8015610db35780601f10610d8a57610100808354040283529160200191610db3565b820191905f5260205f20905b815481529060010190602001808311610d9657829003601f168201915b505050505090806001018054610dc890611598565b80601f0160208091040260200160405190810160405280929190818152602001828054610df490611598565b8015610e3f5780601f10610e1657610100808354040283529160200191610e3f565b820191905f5260205f20905b815481529060010190602001808311610e2257829003601f168201915b505050505090806002018054610e5490611598565b80601f0160208091040260200160405190810160405280929190818152602001828054610e8090611598565b8015610ecb5780601f10610ea257610100808354040283529160200191610ecb565b820191905f5260205f20905b815481529060010190602001808311610eae57829003601f168201915b505050505090806003015490806004015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff16908060040160149054906101000a900460ff16905086565b5f80fd5b5f80fd5b5f819050919050565b610f2d81610f1b565b8114610f37575f80fd5b50565b5f81359050610f4881610f24565b92915050565b5f73ffffffffffffffffffffffffffffffffffffffff82169050919050565b5f610f7782610f4e565b9050919050565b610f8781610f6d565b8114610f91575f80fd5b50565b5f81359050610fa281610f7e565b92915050565b5f8060408385031215610fbe57610fbd610f13565b5b5f610fcb85828601610f3a565b9250506020610fdc85828601610f94565b9150509250929050565b5f80fd5b5f80fd5b5f80fd5b5f8083601f84011261100757611006610fe6565b5b8235905067ffffffffffffffff81111561102457611023610fea565b5b6020830191508360018202830111156110405761103f610fee565b5b9250929050565b5f805f6040848603121561105e5761105d610f13565b5b5f61106b86828701610f3a565b935050602084013567ffffffffffffffff81111561108c5761108b610f17565b5b61109886828701610ff2565b92509250509250925092565b6110ad81610f6d565b82525050565b5f81519050919050565b5f82825260208201905092915050565b8281835e5f83830152505050565b5f601f19601f8301169050919050565b5f6110f5826110b3565b6110ff81856110bd565b935061110f8185602086016110cd565b611118816110db565b840191505092915050565b5f8115159050919050565b61113781611123565b82525050565b5f6080820190506111505f8301876110a4565b818103602083015261116281866110eb565b9050611171604083018561112e565b61117e606083018461112e565b95945050505050565b5f805f805f805f6080888a0312156111a2576111a1610f13565b5b5f88013567ffffffffffffffff8111156111bf576111be610f17565b5b6111cb8a828b01610ff2565b9750975050602088013567ffffffffffffffff8111156111ee576111ed610f17565b5b6111fa8a828b01610ff2565b9550955050604088013567ffffffffffffffff81111561121d5761121c610f17565b5b6112298a828b01610ff2565b9350935050606061123c8a828b01610f3a565b91505092959891949750929550565b61125481610f1b565b82525050565b5f60208201905061126d5f83018461124b565b92915050565b5f6020828403121561128857611287610f13565b5b5f61129584828501610f3a565b91505092915050565b5f6020820190506112b15f8301846110a4565b92915050565b5f60c0820190508181035f8301526112cf81896110eb565b905081810360208301526112e381886110eb565b905081810360408301526112f781876110eb565b9050611306606083018661124b565b61131360808301856110a4565b61132060a083018461112e565b979650505050505050565b7f4f6e6c792074686520656d706c6f7965722063616e20657865637574652074685f8201527f6973000000000000000000000000000000000000000000000000000000000000602082015250565b5f6113856022836110bd565b91506113908261132b565b604082019050919050565b5f6020820190508181035f8301526113b281611379565b9050919050565b7f4170706c69636174696f6e20646f6573206e6f742065786973740000000000005f82015250565b5f6113ed601a836110bd565b91506113f8826113b9565b602082019050919050565b5f6020820190508181035f83015261141a816113e1565b9050919050565b5f6040820190506114345f83018561124b565b61144160208301846110a4565b9392505050565b7f496e7465726e73686970206973206e6f206c6f6e6765722061637469766500005f82015250565b5f61147c601e836110bd565b915061148782611448565b602082019050919050565b5f6020820190508181035f8301526114a981611470565b9050919050565b7f596f75206861766520616c7265616479206170706c69656420666f72207468695f8201527f7320696e7465726e736869700000000000000000000000000000000000000000602082015250565b5f61150a602c836110bd565b9150611515826114b0565b604082019050919050565b5f6020820190508181035f830152611537816114fe565b9050919050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52604160045260245ffd5b7f4e487b71000000000000000000000000000000000000000000000000000000005f52602260045260245ffd5b5f60028204905060018216806115af57607f821691505b6020821081036115c2576115c161156b565b5b50919050565b5f819050815f5260205f209050919050565b5f6020601f8301049050919050565b5f82821b905092915050565b5f600883026116247fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff826115e9565b61162e86836115e9565b95508019841693508086168417925050509392505050565b5f819050919050565b5f61166961166461165f84610f1b565b611646565b610f1b565b9050919050565b5f819050919050565b6116828361164f565b61169661168e82611670565b8484546115f5565b825550505050565b5f90565b6116aa61169e565b6116b5818484611679565b505050565b5b818110156116d8576116cd5f826116a2565b6001810190506116bb565b5050565b601f82111561171d576116ee816115c8565b6116f7846115da565b81016020851015611706578190505b61171a611712856115da565b8301826116ba565b50505b505050565b5f82821c905092915050565b5f61173d5f1984600802611722565b1980831691505092915050565b5f611755838361172e565b9150826002028217905092915050565b61176e826110b3565b67ffffffffffffffff8111156117875761178661153e565b5b6117918254611598565b61179c8282856116dc565b5f60209050601f8311600181146117cd575f84156117bb578287015190505b6117c5858261174a565b86555061182c565b601f1984166117db866115c8565b5f5b82811015611802578489015182556001820191506020850194506020810190506117dd565b8683101561181f578489015161181b601f89168261172e565b8355505b6001600288020188555050505b505050505050565b828183375f83830152505050565b5f61184d83856110bd565b935061185a838584611834565b611863836110db565b840190509392505050565b5f6080820190506118815f83018961124b565b8181036020830152611894818789611842565b905081810360408301526118a9818587611842565b90506118b860608301846110a4565b979650505050505050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52601160045260245ffd5b5f6118fa82610f1b565b91507fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff820361192c5761192b6118c3565b5b600182019050919050565b7f4170706c69636174696f6e206e6f7420617070726f76656400000000000000005f82015250565b5f61196b6018836110bd565b915061197682611937565b602082019050919050565b5f6020820190508181035f8301526119988161195f565b9050919050565b5f81905092915050565b50565b5f6119b75f8361199f565b91506119c2826119a9565b5f82019050919050565b5f6119d6826119ac565b9150819050919050565b7f4661696c656420746f2073656e642073746970656e64000000000000000000005f82015250565b5f611a146016836110bd565b9150611a1f826119e0565b602082019050919050565b5f6020820190508181035f830152611a4181611a08565b9050919050565b5f606082019050611a5b5f83018661124b565b611a6860208301856110a4565b611a75604083018461124b565b94935050505056fea264697066735822122004b5b9f4be70347e97d161dc99e9ec1c8422678627e48e08a4e035709267b30764736f6c634300081a0033', 'value': '0', 'max_priority_fee_per_gas': '100000000', 'revert_reason': None, 'confirmation_duration': [0, 28104.0], 'tx_tag': None}
